# TECHIN 509 – Melody Generator with Files & Testing (Final Version)

This notebook is designed to run **directly inside your `04_Files` folder** with the
following structure:

```text
04_Files/
├── dataset/
│   ├── note_seq_w_dur/
│   │   ├── 000_10_YardFight_00_01_melody.txt
│   │   ├── ... many .txt files
│   └── note_seq_wo_dur/
│       ├── 000_10_YardFight_00_01_melody.txt
│       ├── ... many .txt files
└── techin509_melody_files_and_tests_final.ipynb
```

As long as your notebook file and the `dataset/` folder sit next to each other like above,
all paths in this notebook will work **without any manual changes**.

This notebook implements:

- `load_melodies(path: str) -> list[list[str]]` — load melodies from a **single .txt file**.
- `save_melodies(melodies, path)` — save melodies to a text file.
- Robust file reading with `try/except` (prints a helpful message if file is missing).
- `load_all_melodies_from_folder(folder_path)` — convenience helper to load all `.txt` files in a folder.
- At least **3 unit tests** using `unittest`.


In [ ]:
from __future__ import annotations
from pathlib import Path
from typing import List


def load_melodies(path: str) -> list[list[str]]:
    """Read melodies from a file and return as list of note lists.

    Each line in the file is treated as **one melody**.
    Notes on a line are separated by whitespace (spaces or tabs).

    If the file does not exist, this function prints a helpful message
    and returns an empty list instead of raising an error.
    """

    file_path = Path(path)
    melodies: list[list[str]] = []

    try:
        with file_path.open("r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                notes = line.split()
                melodies.append(notes)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        print("Please make sure the dataset file exists.")

    return melodies


def save_melodies(melodies: list[list[str]], path: str) -> None:
    """Save a list of generated melodies to a file, one melody per line.

    Each melody (a list of note strings) is written as a single line
    with notes separated by a single space.
    """

    file_path = Path(path)
    if file_path.parent and not file_path.parent.exists():
        file_path.parent.mkdir(parents=True, exist_ok=True)

    with file_path.open("w", encoding="utf-8") as f:
        for melody in melodies:
            line = " ".join(melody)
            f.write(line + "\n")


def load_all_melodies_from_folder(folder_path: str) -> list[list[str]]:
    """Load melodies from **all .txt files** inside a folder.

    This is a convenience helper for your NES dataset, where you have
    many small `.txt` files in `note_seq_w_dur/` or `note_seq_wo_dur/`.
    """

    folder = Path(folder_path)
    all_melodies: list[list[str]] = []

    if not folder.exists():
        print(f"Folder not found: {folder}")
        print("Please make sure the dataset folder exists.")
        return []

    for txt_file in sorted(folder.glob("*.txt")):
        melodies = load_melodies(str(txt_file))
        all_melodies.extend(melodies)

    return all_melodies


## Demo: loading from your `dataset/` structure

This cell assumes your current working directory is the folder that contains
both the notebook and the `dataset/` folder (your `04_Files` directory).

It will:

1. Locate `dataset/note_seq_w_dur/` and `dataset/note_seq_wo_dur/`.
2. Automatically pick the **first .txt file** in the `note_seq_w_dur` folder.
3. Load melodies from that file using `load_melodies`.
4. Load melodies from **all files** in `note_seq_w_dur` using `load_all_melodies_from_folder`.


In [ ]:
BASE_DIR = Path('.')  # folder where this notebook lives (e.g., 04_Files)
DATASET_DIR = BASE_DIR / 'dataset'
W_DUR_DIR = DATASET_DIR / 'note_seq_w_dur'
WO_DUR_DIR = DATASET_DIR / 'note_seq_wo_dur'

print("BASE_DIR:", BASE_DIR.resolve())
print("DATASET_DIR exists:", DATASET_DIR.exists())
print("note_seq_w_dur exists:", W_DUR_DIR.exists())
print("note_seq_wo_dur exists:", WO_DUR_DIR.exists())

# 1) Pick the first .txt file from note_seq_w_dur
example_files = sorted(W_DUR_DIR.glob('*.txt'))
if not example_files:
    print("No .txt files found in", W_DUR_DIR)
else:
    example_file = example_files[0]
    print("\nExample file:", example_file)
    example_melodies = load_melodies(str(example_file))
    print(f"Loaded {len(example_melodies)} melodies from example file.")
    if example_melodies:
        print("First melody (first 10 notes):", example_melodies[0][:10])

    # 2) Load ALL melodies from note_seq_w_dur
    all_melodies_w_dur = load_all_melodies_from_folder(str(W_DUR_DIR))
    print(f"\nTotal melodies loaded from note_seq_w_dur: {len(all_melodies_w_dur)}")


## Unit tests with `unittest`

The tests below check that:

1. `load_melodies` correctly loads melodies from a small temporary file.
2. `save_melodies` writes melodies in the correct format and they can be loaded back.
3. `load_melodies` handles a missing file gracefully (prints a message and returns an empty list).

These tests **do not depend on your dataset folder**, so they will run anywhere.


In [ ]:
import io
import tempfile
import unittest
from contextlib import redirect_stdout


class TestMelodyFileIO(unittest.TestCase):
    """Tests for load_melodies and save_melodies."""

    def test_load_melodies_basic(self) -> None:
        """load_melodies should parse lines into lists of note strings."""
        with tempfile.TemporaryDirectory() as tmpdir:
            tmp_path = Path(tmpdir) / 'melodies.txt'
            content = 'C4 D4 E4\nA3 B3 C4 D4\n'  # two melodies
            tmp_path.write_text(content, encoding='utf-8')

            melodies = load_melodies(str(tmp_path))

            self.assertEqual(len(melodies), 2)
            self.assertEqual(melodies[0], ['C4', 'D4', 'E4'])
            self.assertEqual(melodies[1], ['A3', 'B3', 'C4', 'D4'])

    def test_save_and_reload_round_trip(self) -> None:
        """save_melodies followed by load_melodies should give the same data."""
        original = [['C4', 'D4'], ['E4', 'F4', 'G4']]

        with tempfile.TemporaryDirectory() as tmpdir:
            tmp_path = Path(tmpdir) / 'generated.txt'

            save_melodies(original, str(tmp_path))
            loaded = load_melodies(str(tmp_path))

            self.assertEqual(loaded, original)

    def test_load_missing_file_returns_empty_and_prints_message(self) -> None:
        """If file is missing, load_melodies prints a message and returns []."""
        with tempfile.TemporaryDirectory() as tmpdir:
            missing_path = Path(tmpdir) / 'does_not_exist.txt'

            buf = io.StringIO()
            with redirect_stdout(buf):
                result = load_melodies(str(missing_path))

            output = buf.getvalue()
            self.assertEqual(result, [])
            self.assertIn('File not found', output)
            self.assertIn('Please make sure the dataset file exists.', output)


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


## (Optional) Create `tests/test_models.py` for a regular Python project

If your instructor wants a separate `tests/` folder with `test_models.py`,
you can run the cell below **once**. It will create:

```text
tests/
└── test_models.py
```

You may need to adjust the import line depending on where you put
`load_melodies` and `save_melodies` (e.g., in `main.py` or `melody_io.py`).
